# Collecte de données


In [29]:
import os
import pandas as pd

# === Récupération du type de fichier image associé à chaque pokémon ===
lst_files = os.listdir('images/images')
lst_files = [f.split('.') for f in lst_files]
dico_files = {paire[0]: paire[1] for paire in lst_files}

#importation des données depuis un fichier csv
with open('images/pokemon.csv', 'r') as csv_file:

    # === Pré-traitement des données ===
    lines = csv_file.readlines()
    lines = lines[1:] #suppression de la première ligne (entête)
    
    for i,data in enumerate(lines):
        data = data.replace('\n', '')
        lines[i] = data.split(',')
        if len(lines[i]) == 2: #si le pokémon n'a qu'un seul type
            lines[i].append(None) #ajout d'une colonne vide pour le second type
        #récupération du type de fichier image associé   
        lines[i].append(dico_files[lines[i][0]])
        
    # === Création d'un DataFrame ===      
    df = pd.DataFrame(lines, columns=["name", "type1", "type2","img_type"])
    
    csv_file.close()

# Export du DataFrame vers un fichier JSON
df.to_json('donnees.json',orient='index')

# Étiquetage et annotation

In [32]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans

imgfile = Image.open("images/images/pichu.png")

numarray = numpy.array(imgfile, numpy.uint8)

numarray = numarray.reshape(-1, 9)

n_clusters = 4
clusters = KMeans(n_clusters=n_clusters, n_init=20)
clusters.fit(numarray)

couleurs = []
for i in range(len(clusters.cluster_centers_)):
    print(clusters.cluster_centers_[i])
    couleurs.append([numpy.sum(clusters.labels_==i),"#%02x%02x%02x" % (
        math.ceil(clusters.cluster_centers_[i][0]),
        math.ceil(clusters.cluster_centers_[i][1]),
        math.ceil(clusters.cluster_centers_[i][2]),
    )])
    
print (couleurs)

[1.29876374 0.53708791 0.18269231 0.11401099 0.04052198 0.04052198
 0.09134615 0.2967033  0.77197802]
[142.86764706 157.57352941 171.20588235 177.23529412 181.19117647
 182.29411765 173.25       166.98529412 156.63235294]
[145.175 144.25  135.925 113.15   89.225  64.05   40.1    24.025  19.45 ]
[  4.           7.69444444  20.63888889  35.11111111  63.05555556
  94.44444444 115.55555556 141.55555556 146.05555556]
[[1456, '#020101'], [68, '#8f9eac'], [40, '#929188'], [36, '#050815']]


# Analyses de données

In [18]:
import ipywidgets as widgets
from PIL import Image
import os
import json
import random
from IPython.display import display

bouton_jaime = None
bouton_jaime_pas = None

def afficher_image_aleatoire(b):
    global bouton_jaime, bouton_jaime_pas

    with open("donnees.json", "r") as f:
        donnees = json.load(f)

    image_id = str(random.randint(0, len(donnees) - 1))
    nom_image = donnees[image_id]["name"]
    img_type = donnees[image_id]["img_type"]

    chemin_image = os.path.join("images/images", f"{nom_image}.{img_type}")
    image = Image.open(chemin_image)

    image_widget.value = open(chemin_image, "rb").read()

    label_nom_image.value = nom_image

    # Supprimer les anciens boutons s'ils existent
    if bouton_jaime:
        bouton_jaime.close()
    if bouton_jaime_pas:
        bouton_jaime_pas.close()

    bouton_jaime = widgets.Button(description="J'aime")
    bouton_jaime.on_click(lambda b: action_jaime(image_id))
    bouton_jaime_pas = widgets.Button(description="Je n'aime pas")
    bouton_jaime_pas.on_click(lambda b: action_jaime_pas(image_id))

    display(widgets.HBox([bouton_jaime, bouton_jaime_pas]))

def remplacement_bouton(b):
    bouton_afficher_image.close()
    afficher_image_aleatoire(b)

def action_jaime(image_id):
    modifier_preferences(image_id, 1)
    afficher_image_aleatoire(None)

def action_jaime_pas(image_id):
    modifier_preferences(image_id, -0.5)
    afficher_image_aleatoire(None)

def modifier_preferences(image_id, valeur):
    with open("donnees.json", "r+") as f:
        donnees = json.load(f)
        donnees[str(image_id)]["preferences"] = valeur
        f.seek(0)
        json.dump(donnees, f, indent=4)
        f.truncate()

image_widget = widgets.Image(format='png')
display(image_widget)

label_nom_image = widgets.Label(value="")
display(label_nom_image)

bouton_afficher_image = widgets.Button(description="Afficher une image")
bouton_afficher_image.on_click(remplacement_bouton)
display(bouton_afficher_image)


Image(value=b'')

Label(value='')

Button(description='Afficher une image', style=ButtonStyle())

# Visualisation des données

In [ ]:
pass

# Système de recommandation

In [ ]:
pass

# Tests

pass